# **Table of Contents**

    * Iteration 2 - Homework
        - Data Definitions
        - Column Titles
        - Sample Data
        - Format the data
        - Questions
        - Predictive Models
    * Demo

 
# **Iteration 2 - Homework**

 
## **Data Definitions**

The following real-world Uverse data gives information about the viewers of several TV shows. The information included about a particular view is gender, age, household income code, and observation status of a given TV show. A positive observation (‘1’) indicates that the particular viewer watched the given show, while a negative observation (‘0’) indicates they do not watch the show. There are six shows captured for each viewer. Each field is separated by a pipe (‘|’). The data is available here.

 
## **Column Titles**

PERSON_GENDER_CD | PERSON_AGE_NBR | HOUSEHOLD_INCOME_ID | DOG_WHISPERER | DOGS_101 | CATS_101 | BAD_DOG | PUPPIES_VS_BABIES | PUPPY_BOWL

 
## **Sample Data**

1|47|C|1|0|0|0|0|0

1|51|D|0|0|0|0|0|0

1|26|9|0|0|0|0|0|0

 
## **Format the data**

### **Gender Code**

Valid Gender Codes:

1-3 where

        * 1 = Male

        * 2 = Female

        * 3 = Unknown

All other values in gender code should be cleaned up by either removing them or adding them to Unknown


### **Income Level Code**

Valid Income Level Codes:

    * 0-9, A-D

Invalid Income Level Codes:

    * NA


### **TV Shows**

Valid values:

    * 1 = Positive Observation

    * 0 = Negative Observation


## **Questions**

    1 How many viewers have the income code 4?

    2 How many females are in the population?

    3 What is the median age?

    4 What is the mode of the income level code?

    5 How many viewers are under the age of 18?


## **Predictive Models**

* Regression

    Create a linear model to showing the relationship between a viewer’s age and the total number of TV shows watched.


* Classification / Clustering

    Create a cluster about each show (might be best to do a subset for each TV show) to show which age/age range is most likely to watch that show.
    Create a cluster showing which gender code is more likely to have a certain income level. Hint: you can take a similar approach to question 1 for the graphs or you can find a way to put all income codes in 1 graph.

 
## **Demo**

    Congratulations on completing the iteration tasks in preparation for your demo.
    Please review the Demo 1-Pager to help ensure you are fully prepared.
    Attend and complete the demo previously scheduled with your Topic Advisor

In [3]:
mud <- read.csv(rcloud.home("modified_uverse_data.txt", user="jl2408"), sep="|")

In [4]:
colnames(mud) <- c("gender", "age", "income", "dog_whisperer", "dogs_101", "cats_101", "bad_dog", "puppies_vs_babies", "puppy_bowl")
head(mud)
str(mud)

How many viewers have the income code 4?

In [6]:
table(mud$income)

How many females are in the population?

In [8]:
table(mud$gender)
summary(mud$gender)

What is the median age?

In [10]:
table(mud$age)
summary(mud$age)
median(mud$age,  na.rm=TRUE)
# 

In [11]:
library("ggplot2")
qplot(x = age, data = mud, binwidth = 1, color = I('black'), fill = I('orange'))
# 
qplot(x = age, data = subset(mud, gender != "*" & gender != 3 & !is.na(gender) &!is.na(age)), color = I('black'), fill = gender, xlab = "Age", ylab = "Number of viewers") + facet_wrap(~gender)
# 

What is the mode of the income level code?

In [13]:
summary(mud$income)
ta <- table(mud$income)
ta
tam <- max(table(mud$income))
tam
names(ta)[ta == tam]
# 
# 

How many viewers are under the age of 18?

In [15]:
dim(mud)
mud.complete <- mud[complete.cases(mud$age),]
dim(mud.complete)
sum(mud$age < 18, na.rm=TRUE)
dim(mud.complete[mud.complete$age < 18,])

In [16]:
summary(mud$gender)
table(mud$gender)
summary(mud$age)
table(mud$age)
summary(mud$income)
table(mud$income)
summary(mud$dog_whisperer)
summary(mud$dogs_101)
summary(mud$cats_101)
summary(mud$bad_dog)
summary(mud$puppies_vs_babies)
summary(mud$puppy_bowl)

Create a linear model to showing the relationship between a viewer’s age and the total number of TV shows watched.
we need to clean invalid data in age

In [18]:
mud.age.complete <- mud[complete.cases(mud$age),]
# mud.age.valid <- subset(mud.age.complete, age > 0)
# str(mud.age.valid)

Add a new column for total number of TV shows watched- shows_watched

In [20]:
mud.age.complete$shows_watched <- rowSums(mud.age.complete[, c("dog_whisperer", "dogs_101", "cats_101", "bad_dog", "puppies_vs_babies", "puppy_bowl")])
str(mud.age.complete)

Plot summary stat on top of the scattered plot
mean: solid black
10% quantile: blue
90% quantile: green
we see most people do not watch shows

In [22]:
library(ggplot2)
ggplot(aes(x = age, y = shows_watched), data = mud.age.complete) + geom_point(alpha = 1/20, color = 'orange') +
geom_line(stat = 'summary', fun.y = mean) +
geom_line(stat = 'summary', fun.y = quantile, fun.args=list(probs = .1), linetype = 2, color = 'blue') + 
geom_line(stat = 'summary', fun.y = quantile, fun.args=list(probs = .9), linetype = 2, color = 'green')

Plot summary stat on top of the scattered plot
Adjust coordinate to show summary stats better
mean: solid black
10% quantile: blue
90% quantile: green
we see most people do not watch shows

In [24]:
library(ggplot2)
ggplot(aes(x = age, y = shows_watched), data = mud.age.complete) + 
coord_cartesian(ylim = c(0,2)) +
geom_point(alpha = 1/20, color = 'orange') +
geom_line(stat = 'summary', fun.y = mean) +
geom_line(stat = 'summary', fun.y = quantile, fun.args=list(probs = .1), linetype = 2, color = 'blue') + 
geom_line(stat = 'summary', fun.y = quantile, fun.args=list(probs = .9), linetype = 2, color = 'green')
# Evaluate correlation between age and shows_watched
# Observation: no correlation
# plot the relationship between age and show_cnt

In [25]:
library(dplyr)
# 
shows_watched_by_age <- mud.age.complete %>% group_by(age) %>% summarise(shows_watched_mean = mean(shows_watched), shows_watched_median = median(shows_watched), viewers = n())
# 
str(shows_watched_by_age)
head(shows_watched_by_age)
with(shows_watched_by_age, cor.test(age, shows_watched_mean, method = 'pearson'))
ggplot(aes(age, shows_watched_mean), data = shows_watched_by_age) + geom_point()

In [26]:
# age_shows_watched_per_viewer <- mud.age.complete %>% select(age, shows_watched) %>% group_by(age) %>% summarise(shows_watched_by_age = sum(shows_watched)/n())
# 
# 
testidx <- which(1:nrow(shows_watched_by_age)%%4==0)
shows_watched_by_age.train <- shows_watched_by_age[-testidx,]
shows_watched_by_age.test <- shows_watched_by_age[testidx,]
lr.model <- lm(formula = shows_watched_mean ~ age, data = shows_watched_by_age.train)
summary(lr.model)

In [27]:
plot(shows_watched_mean~age, shows_watched_by_age)
abline(lr.model)

Create a cluster about each show (might be best to do a subset for each TV show) to show which age/age range is most likely to watch that show.

```{r}
# library(gridExtra)
# 
# dog_whisperer watched
# cluster centered at 50 is most likely to watch dog_whisperer
dog_whisperer <- mud.age.complete %>% group_by(age) %>% summarise(watched_per_viewer = mean(dog_whisperer))
clusters <- kmeans(dog_whisperer,4)
clusters$size
clusters$centers
dog_whisperer$cluster <- clusters$cluster
qplot(x = age, y = watched_per_viewer, data = dog_whisperer, color = factor(cluster), main="dog_whisperer watched by age")
# 
# dogs_101 watched
# cluster centered at 48.5 is most likely to watch dogs_101
dogs_101 <- mud.age.complete %>% group_by(age) %>% summarise(watched_per_viewer = mean(dogs_101))
clusters <- kmeans(dogs_101,4)
clusters$size
clusters$centers
dogs_101$cluster <- clusters$cluster
qplot(x = age, y = watched_per_viewer, data = dogs_101, color = factor(cluster), main="dogs_101 watched by age")
# 
# cats_101 watched
# cluster centered at 72.5 is most likely to watch cats_101
cats_101 <- mud.age.complete %>% group_by(age) %>% summarise(watched_per_viewer = mean(cats_101))
clusters <- kmeans(cats_101,4)
clusters$size
clusters$centers
cats_101$cluster <- clusters$cluster
qplot(x = age, y = watched_per_viewer, data = cats_101, color = factor(cluster), main="cats_101 watched by age")
# 
# 
# bad_dog watched
# cluster centered at 72 is most likely to watch bad_dog
bad_dog <- mud.age.complete %>% group_by(age) %>% summarise(watched_per_viewer = mean(bad_dog))
clusters <- kmeans(bad_dog,4)
clusters$size
clusters$centers
bad_dog$cluster <- clusters$cluster
qplot(x = age, y = watched_per_viewer, data = bad_dog, color = factor(cluster), main="bad_dog watched by age")
# 
# 
# puppies_vs_babies watched
# cluster centered at 94.5 is most likely to watch puppies_vs_babies
puppies_vs_babies <- mud.age.complete %>% group_by(age) %>% summarise(watched_per_viewer = mean(puppies_vs_babies))
clusters <- kmeans(puppies_vs_babies,4)
clusters$size
clusters$centers
puppies_vs_babies$cluster <- clusters$cluster
qplot(x = age, y = watched_per_viewer, data = puppies_vs_babies, color = factor(cluster), main="puppies_vs_babies watched by age")
# 
# puppy_bowl watched
# cluster centered at 72.5 is most likely to watch puppy_bowl
puppy_bowl <- mud.age.complete %>% group_by(age) %>% summarise(watched_per_viewer = mean(puppy_bowl))
clusters <- kmeans(puppy_bowl,4)
clusters$size
clusters$centers
puppy_bowl$cluster <- clusters$cluster
qplot(x = age, y = watched_per_viewer, data = puppy_bowl, color = factor(cluster), main="puppy_bowl watched by age")
```
grid.arrange(p1,p2,p3,p4,p5,p6,ncol=1)
Create a cluster showing which gender code is more likely to have a certain income level. Hint: you can take a similar approach to question 1 for the graphs or you can find a way to put # all income codes in 1 graph.
Explore income distribution

Observations:
* 1. more female than male have income indicator from 1 to 6
* 2. more male than female have income indicator from 7 to d

In [30]:
ss <- subset(mud, gender != "*" & gender != 3 & !is.na(gender) &!is.na(income))
ss$gender <- as.character(ss$gender)
ss$gender[ss$gender ==1] <- "Male"
ss$gender[ss$gender ==2] <- "Female"
ss$gender <- as.factor(ss$gender)
# 
ggplot(ss, aes(income)) + geom_bar(aes(fill=..count..)) + scale_fill_gradient("Count", low = "green", high = "red") + labs( title = "Income Histogram", y = "Count", x = "Income Indicator")
# 
ggplot(ss, aes(x=income, fill=gender)) + geom_bar(position="dodge") + labs( title = "Income Histogram", y = "Count", x = "Income Indicator")

In [31]:
qplot(x = income, data = ss, color = I('black'), fill = gender, xlab = "Income Indicator", ylab = "Number of viewers") + facet_wrap(~gender)

In [32]:
men = c(100, 120, 60)
women = c(350, 200, 90)
ice.cream.survey = as.data.frame(rbind(men, women))
names(ice.cream.survey) = c('chocolate', 'vanilla', 'strawberry')
ice.cream.survey
chisq.test(ice.cream.survey)

In [33]:
# chi-squared test of independence - gender / income
# reject null
by(ss$income, ss$gender,summary)
a <- as.data.frame.matrix(table(ss$gender, ss$income))
a
chisq.test(a)
# 
# 

we need to clean invalid data in age
other tests

In [35]:
# 
# remove NA in gender
mud.gender.income.complete <- mud[complete.cases(mud$gender),]
# 
table(mud.gender.income.complete$gender)
table(mud.gender.income.complete$income)
# 
# remove NA in income
mud.gender.income.complete <- mud.gender.income.complete[complete.cases(mud.gender.income.complete$income),]
# 
table(mud.gender.income.complete$gender)
table(mud.gender.income.complete$income)
# 
levels(mud.gender.income.complete$gender)
levels(mud.gender.income.complete$income)
# 
# construct a subset of gender and income
mud.gender.income.valid <- mud.gender.income.complete %>% select(gender, income)
# 
# convert factor to charactor
mud.gender.income.valid$gender <- as.character(mud.gender.income.valid$gender )
mud.gender.income.valid$income <- as.character(mud.gender.income.valid$income)
# 
# 
# remove unknown and invalid gender code
mud.gender.income.valid <- mud.gender.income.valid[mud.gender.income.valid$gender %in% c("1","2"),]
# 
# 
table(mud.gender.income.valid$gender)
table(mud.gender.income.valid$income)
# 
# covert charactor to number
mud.gender.income.valid$income[mud.gender.income.valid$income == "A"] <- "10"
mud.gender.income.valid$income[mud.gender.income.valid$income == "B"] <- "11"
mud.gender.income.valid$income[mud.gender.income.valid$income == "C"] <- "12"
mud.gender.income.valid$income[mud.gender.income.valid$income == "D"] <- "13"
# 
# Convert to factor
mud.gender.income.valid$gender <- as.factor(mud.gender.income.valid$gender )
# mud.gender.income.valid$income <- as.factor(mud.gender.income.valid$income)
# Convert to integer
mud.gender.income.valid$income <- as.integer(mud.gender.income.valid$income)
# 
table(mud.gender.income.valid$gender)
table(mud.gender.income.valid$income)
# 
# 
str(mud.gender.income.valid)
summary(mud.gender.income.valid)
table(mud.gender.income.valid$gender)
table(mud.gender.income.valid$income)

Histogram

In [37]:
library(ggplot2)
ggplot(mud.gender.income.valid, aes(income)) + geom_histogram(breaks = seq(1,13), aes(fill=..count..)) + scale_fill_gradient("Count", low = "green", high = "red") + labs( title = "Income Histogram", y = "Count", x = "Income Indicator")

In [38]:
ggplot(mud.gender.income.valid, aes(x=income, fill=gender)) + geom_histogram(binwidth=0.5, position="dodge") + labs( title = "Income Histogram", y = "Count", x = "Income Indicator")

In [39]:
clusters <- kmeans(mud.gender.income.valid,2)
clusters$size
clusters$centers
# 
# create a new column in the mud.gender.income.valid dataframe for the cluster value
mud.gender.income.valid$cluster <- clusters$cluster
# 
plot(income~gender, mud.gender.income.valid, pch=21, bg=c("red", "green") [unclass(cluster)])
# 

In [40]:
library(tidyr)
df <- gather(mud, "shows", "watched", 4:9)
str(df)
summary(df)
# 

In [41]:
table(df$shows)
by(df$watched, df$shows, sum)